In [1]:
from src.nodes.understand_element_nodes import agent_preprocessor, agent_code_generation, agent_extract_code, agent_code_review
from src.nodes.understand_element_nodes import conditional_should_continue_after_extraction, conditional_should_continue_after_code_review
import os, io, json, docker, tarfile

In [2]:
"""class AgentState(TypedDict):
    file_path: str
    first_few_lines: str
    user_info_about_data: str
    first_few_elements_path_name: str
    extracted_python_code: str
    code_review_result: CodeReviewResult
    generated_code: str
    instructions_to_code: str
    code_extraction_status: str
    code_review_status: str"""

init_state = {
    "file_path": "data.json",
    "first_few_lines": "",
    "user_info_about_data": "",
    "first_few_elements_path_name": "elements.json",
    "extracted_python_code": "",
    "code_review_result": "",
    "generated_code": "",
    "instructions_to_code": "",
    "code_extraction_status": "",
    "code_review_status": ""
}

In [3]:
#data.json is very large, so only first few lines are loaded. load only 10% of the file
def load_data(file_path):
    with open('data.json', 'r') as file:
        chunk = file.read(1024)
    return chunk

In [4]:
init_state["first_few_lines"] = load_data(init_state["file_path"])

In [5]:
init_state["user_info_about_data"] = "This is a json file. It contains a list of dictionaries. Elements of list are minecraft events triggered by players in minecraft server."

In [6]:
init_state = agent_preprocessor(init_state)

DEBUG: Preprocessing User Request...
DEBUG: Preprocessor Result: Here are the detailed instructions to achieve the required task using Python:

1. **Import Necessary Modules**: Begin by importing the necessary Python modules. You'll need `json` for handling JSON data and `os` for handling file paths.

2. **Define the File Paths**: Use the `os.path` module to define the paths for the input and output files. This ensures that the file paths are handled correctly across different operating systems.

3. **Read the JSON File**: Open the JSON file located at `data.json` and read its contents. Use a `with` statement to ensure that the file is closed properly after reading.

4. **Parse the JSON Data**: Load the contents of the file into a Python object using `json.load()`. This will convert the JSON data into a list of dictionaries.

5. **Extract the First Few Elements**: Determine how many elements you want to extract. For example, you can extract the first 5 elements. Make sure to handle the

In [7]:
temp = init_state.copy()
temp = agent_code_generation(temp)

DEBUG: Generating Python Code...
DEBUG: Initial Generation of code. No need to reset agent state.
DEBUG: Code Generation Result: ```
import json
import os

def extract_and_save_elements(input_filename, output_filename, num_elements):
    base_path = os.getcwd()
    input_file_path = os.path.join(base_path, input_filename)
    output_file_path = os.path.join(base_path, output_filename)

    with open(input_file_path, 'r') as file:
        data = json.load(file)

    extracted_elements = data[:num_elements]

    with open(output_file_path, 'w') as file:
        json.dump(extracted_elements, file, indent=4)

extract_and_save_elements('data.json', 'elements.json', 5)
```
DEBUG: agent_code_generation state
--------------------------------------------------
Agent State:
file_path: data.json
first_few_lines: [{"event": "PlayerJoinEvent", "timestamp": "2023-12-17 06:42:09", "time": 1702775529, "distinct_id": "00000000-0000-0000-0009-01fff2bbdb48", "$city": "Mumbai", "$insert_id": "tpEsgyuryggd

In [8]:
init_state = temp

In [9]:
temp = init_state.copy()
temp = agent_extract_code(temp)

DEBUG: Extracted Python Code from triple backticks
DEBUG: agent_extract_code state
--------------------------------------------------
Agent State:
file_path: data.json
first_few_lines: [{"event": "PlayerJoinEvent", "timestamp": "2023-12-17 06:42:09", "time": 1702775529, "distinct_id": "00000000-0000-0000-0009-01fff2bbdb48", "$city": "Mumbai", "$insert_id": "tpEsgyuryggdkrvcbesqEbBmvhvburxBFrAg", "$ip": "106.204.62.69", "$mp_api_endpoint": "api.mixpanel.com", "$mp_api_timestamp_ms": 1702775550982, "$region": "Maharashtra", "mp_country_code": "IN", "mp_lib": "jdk", "mp_processing_time_ms": 1702775551040, "server": "manas", "$identified_id": null, "$import": null, "$lib_version": null, "$source": null, "$user_id": null, "event_original_name": null, "id": null, "segment_source_name": null, "type": null, "username": null, "command": null, "full-command": null, "block": null, "world": null, "advancement": null, "victim_name": null, "victim_type": null, "message": null, "to": null, "cause": n

In [10]:
init_state = temp
temp = init_state.copy()

In [11]:
temp = agent_code_review(temp)

DEBUG: Reviewing Python Code...
DEBUG: agent_code_review state
--------------------------------------------------
Agent State:
file_path: data.json
first_few_lines: [{"event": "PlayerJoinEvent", "timestamp": "2023-12-17 06:42:09", "time": 1702775529, "distinct_id": "00000000-0000-0000-0009-01fff2bbdb48", "$city": "Mumbai", "$insert_id": "tpEsgyuryggdkrvcbesqEbBmvhvburxBFrAg", "$ip": "106.204.62.69", "$mp_api_endpoint": "api.mixpanel.com", "$mp_api_timestamp_ms": 1702775550982, "$region": "Maharashtra", "mp_country_code": "IN", "mp_lib": "jdk", "mp_processing_time_ms": 1702775551040, "server": "manas", "$identified_id": null, "$import": null, "$lib_version": null, "$source": null, "$user_id": null, "event_original_name": null, "id": null, "segment_source_name": null, "type": null, "username": null, "command": null, "full-command": null, "block": null, "world": null, "advancement": null, "victim_name": null, "victim_type": null, "message": null, "to": null, "cause": null, "msg": null, "e

In [12]:
import tempfile, shutil

In [73]:
def process_with_docker(init_state):

    client = docker.from_env()

    # Use a safer temporary directory
    custom_temp_base_dir = "temp"
    os.makedirs(custom_temp_base_dir, exist_ok=True)
    temp_dir = tempfile.mkdtemp(dir=custom_temp_base_dir)
    print(f"Temporary directory created at: {temp_dir}")

    container = None
    json_data = None

    # Create data.json
    data_json_path = init_state['file_path']
    destination_data_path = os.path.join(temp_dir, "data.json")
    shutil.copy(data_json_path, destination_data_path)
    print(f"Copied data.json to: {destination_data_path}")

    """# Create elements.json as an empty file
    destination_elements_path = os.path.join(temp_dir, "elements.json")
    with open(destination_elements_path, "w") as f:
        json.dump([], f)
    print(f"Created empty elements.json at: {destination_elements_path}")"""

    # Write the Python script
    temp_script_path = os.path.join(temp_dir, "temp_script.py")
    with open(temp_script_path, "w") as f:
        f.write(init_state['extracted_python_code'])
    print(f"Written script to: {temp_script_path}")

    # Run the Docker container
    container = client.containers.run(
        image="python",
        volumes={os.path.abspath(temp_dir): {'bind': '/usr/src/app', 'mode': 'rw', 'size': '5g'}},
        working_dir="/usr/src/app",
        detach=True,
        mem_limit="5g",
        stdout=True,
        stderr=True
    )
    container = client.containers.get(container.id)
    print(f"Container started: {container.id}")
    exit_code, output = container.exec_run(f"python temp_script.py", workdir="/usr/src/app", privileged=True)
    print("Python script output:", output.decode("utf-8"))
    
    # Debug: List files in the container
    con
    exit_code, output = container.exec_run("ls -l /usr/src/app", privileged=True)
    print("Files in container:", output.decode("utf-8"))

    # Retrieve elements.json
    output_file_path = "/usr/src/app/elements.json"
    tar_stream, _ = container.get_archive(output_file_path)
    tar_bytes = io.BytesIO(b"".join(tar_stream))

    print("Extracted TAR file:", tar_stream)

    # Extract JSON from TAR
    with tarfile.open(fileobj=tar_bytes, mode="r") as tar:
        for member in tar.getmembers():
            if member.name.endswith(".json"):
                json_file = tar.extractfile(member)
                json_data = json.loads(json_file.read().decode("utf-8"))
                print("Extracted JSON data:", json_data)

    return json_data


In [74]:
process_with_docker(init_state)

Temporary directory created at: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpzk_4d7jt
Copied data.json to: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpzk_4d7jt/data.json
Written script to: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpzk_4d7jt/temp_script.py
Container started: 0a5b39be68d831b9dcfcb4a3700b0768b7ab46330588b045bba08c0b11486aa5
Python script output: OCI runtime exec failed: exec failed: unable to start container process: read init-p: connection reset by peer: unknown



NameError: name 'con' is not defined

In [ ]:
custom_temp_base_dir = "temp"
os.makedirs(custom_temp_base_dir, exist_ok=True)
temp_dir = tempfile.mkdtemp(dir=custom_temp_base_dir)
print(f"Temporary directory created at: {temp_dir}")

container = None
json_data = None

# Create data.json
data_json_path = init_state['file_path']
destination_data_path = os.path.join(temp_dir, "data.json")
shutil.copy(data_json_path, destination_data_path)
print(f"Copied data.json to: {destination_data_path}")

# Create elements.json as an empty file
destination_elements_path = os.path.join(temp_dir, "elements.json")
with open(destination_elements_path, "w") as f:
    json.dump([], f)
print(f"Created empty elements.json at: {destination_elements_path}")

# Write the Python script
temp_script_path = os.path.join(temp_dir, "temp_script.py")
with open(temp_script_path, "w") as f:
    f.write(init_state['extracted_python_code'])
print(f"Written script to: {temp_script_path}")

Temporary directory created at: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpkiwlfjae
Copied data.json to: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpkiwlfjae/data.json
Written script to: /Volumes/RenzovPersonal/packages/LiveOpsFrontDashboardMain/backend-testing/agentic-trials/temp/tmpkiwlfjae/temp_script.py


In [39]:
client = docker.from_env()

In [ ]:
#create a container with temporary directory as volume
container = client.containers.run(
    image="python:3.9-slim",
    volumes={temp_dir: {'bind': '/usr/src/app', 'mode': 'rw'}},
    working_dir="/usr/src/app",
    detach=True,
    mem_limit="6g",
    command="python temp_script.py",
    
)

In [62]:
from crewai_tools import CodeInterpreterTool

In [63]:
tool = CodeInterpreterTool()

In [67]:
tool.run_code_in_docker(code="""
import json
import os

def extract_and_save_elements(input_filename, output_filename, num_elements):
    base_path = os.getcwd()
    input_file_path = os.path.join(base_path, input_filename)
    output_file_path = os.path.join(base_path, output_filename)

    with open(input_file_path, 'r') as file:
        data = json.load(file)

    extracted_elements = data[:num_elements]

    print(extracted_elements)

extract_and_save_elements('data.json', 'elements.json', 5)
""", libraries_used=['json', 'os'])

'Something went wrong while running the code: \n'

In [69]:
with open('data.json', 'r') as file:
    elements = json.load(file)

In [70]:
d = []
for i in range(20):
    d.append(elements[i])

In [72]:
with open('data.json', 'w') as file:
    json.dump(d, file)